# RICE-VIRT-DATA-PT-05-2022-U-B-MW Module 17 Challenge

## Code Summary
- **Purpose  :** Evaluation of Credit Risk Machine Learning Ensemble Algorithms 
- **Created  :** 2022 Sept 08 02:39:59 UTC (Meghan E. Hull)
- **Modified :** 2022 Sept 08 03:21:00 UTC (Meghan E. Hull)

## Dependencies

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Version Check

In [4]:
!python --version

Python 3.7.13


In [5]:
!conda list | findstr numpy

numpy                     1.21.5           py37h7a0a035_1  
numpy-base                1.21.5           py37hca35cd5_1  
numpydoc                  1.2                pyhd3eb1b0_0  


In [6]:
!conda list | findstr pandas

pandas                    1.3.5            py37h6214cd6_0  


In [7]:
!conda list | findstr scikit-learn

scikit-learn              1.0.2            py37hf11a4ad_1  
scikit-learn-intelex      2021.5.0         py37haa95532_0  


In [8]:
!conda list | findstr imbalanced-learn

imbalanced-learn          0.9.0                    pypi_0    pypi


## Report Tables

In [9]:
summary_path = Path('Data/Resampling_Summary.csv')

In [12]:
summary_df = pd.read_csv(summary_path,index_col=0)

summary_df

,Balanced Accuracy Score,High Risk Precision Score,Low Risk Precision Score,High Risk Recall Score,Low Risk Recall Score,High Risk F1 Score,Low Risk F1 Score
Naive Random Oversampling,0.628533,0.008839,0.996909,0.597701,0.659364,0.017420,0.793741
SMOTE Oversampling,0.641721,0.009267,0.997100,0.620690,0.662753,0.018262,0.796252
Cluster Centroids Undersampling,0.530070,0.005575,0.995670,0.643678,0.416462,0.011054,0.587281
SMOTEENN Hybrid Sampling,0.651763,0.009116,0.997391,0.678161,0.625365,0.017991,0.768734


# 1. Import & Prep Client Data

## 1.1 Import & Initial Cleaning of Data

In [13]:
# File name
file_path = Path('Data/LoanStats_2019Q1.csv')
file_path

WindowsPath('Data/LoanStats_2019Q1.csv')

In [14]:
# Columns in .csv file
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [15]:
# Load the data
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [16]:
# Check column types
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df.dtypes)

loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_d                        object
loan_status                    object
pymnt_plan                     object
dti                           float64
delinq_2yrs                   float64
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
total_acc                     float64
initial_list_status            object
out_prncp                     float64
out_prncp_inv                 float64
total_pymnt                   float64
total_pymnt_inv               float64
total_rec_prncp               float64
total_rec_int                 float64
total_rec_late_fee            float64
recoveries                    float64
collection_recovery_fee       float64
last_pymnt_a

## 1.2 Split into Training & Testing
The data has been split along the following parameters:
- Target values are in the column "loan_status"
- Features values are converted from strings to numbers using the `get_dummies()` method, excluding the columns:
  - Text columns - "home_ownership", "verification_status", "initial_list_status", "application_type"
  - Date columns - "issue_d", "next_pymnt_d"
  - Boolean / "Y/N" columns - "pymnt_plan", "hardship_flag", "debt_settlement_flag"

In [17]:
# Create target
y = pd.DataFrame(df["loan_status"])

# Create features
X = pd.get_dummies(df, columns=['home_ownership', 
                                'verification_status', 
                                'issue_d', 
                                'pymnt_plan',
                                'initial_list_status', 
                                'next_pymnt_d', 
                                'application_type', 
                                'hardship_flag', 
                                'debt_settlement_flag']).drop('loan_status', axis=1)


In [18]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [19]:
y.head(10)

,loan_status
0,low_risk
1,low_risk
2,low_risk
3,low_risk
4,low_risk
5,low_risk
6,low_risk
7,low_risk
8,low_risk
9,low_risk


In [20]:
# Check the balance of target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [21]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
print('Feature (X) Data Sets')
print(f'X_train Shape: {X_train.shape}')
print(f'X_test Shape: {X_test.shape}')
print('')
print('Target (y) Data Sets')
print(f'y_train Shape: {y_train.shape}')
print(f'y_test Shape: {y_test.shape}')
print('')
print('Target (y) Counters')
print(f'y_train: {Counter(y_train)}')
print(f'y_test: {Counter(y_test)}')

Feature (X) Data Sets
X_train Shape: (51612, 95)
X_test Shape: (17205, 95)

Target (y) Data Sets
y_train Shape: (51612, 1)
y_test Shape: (17205, 1)

Target (y) Counters
y_train: Counter({'loan_status': 1})
y_test: Counter({'loan_status': 1})


# 2. Ensemble Learners
## 2.1 Overview
In this section, two ensemble algorithms are compared to determine which algorithm results in the best performance: the Balanced Random Forest Classifier and an Easy Ensemble AdaBoost Classifier. Each algorithm uses the folliowing steps:

1. Train a logistic regression model with the split data.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

For the Balanced Random Forest Classifier, the feature importance sorted in descending order (most important feature to least important) along with the feature score are also printed.

**Note:** A random state of 1 is used for each sampling algorithm to ensure consistency between tests

## 2.2 Balanced Random Forest Classifier (`BalancedRandomForestClassifier`)

### 2.2.1 Train model & predict test target values

In [23]:
# Train the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

# Instantiate the model
myBRFC = BalancedRandomForestClassifier(random_state=1)

# Train the model using the training data
myBRFC.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [24]:
# Generate predicted target values
y_pred = myBRFC.predict(X_test)

### 2.2.2 Balanced Accuracy Score

In [25]:
# Calculated the balanced accuracy score
modelBAScore = balanced_accuracy_score(y_test, y_pred)
modelBAScore

0.7877672625306695

### 2.2.3 Confusion Matrix

In [26]:
# Display the confusion matrix
myCM = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
myCM_df = pd.DataFrame(
    myCM, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
myCM_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,58,29
Actual low_risk,1560,15558


### 2.2.4 Classification Report

In [27]:
# Print the imbalanced classification report
myReport = classification_report_imbalanced(y_test, y_pred, output_dict=True)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.67      0.91      0.07      0.78      0.59        87
   low_risk       1.00      0.91      0.67      0.95      0.78      0.62     17118

avg / total       0.99      0.91      0.67      0.95      0.78      0.62     17205



In [28]:
summary_df.loc['Balanced Random Forest Ensemble'] = [modelBAScore,
                                               myReport['high_risk']['pre'],
                                               myReport['low_risk']['pre'],
                                               myReport['high_risk']['rec'],
                                               myReport['low_risk']['rec'],
                                               myReport['high_risk']['f1'],
                                               myReport['low_risk']['f1']]
summary_df

,Balanced Accuracy Score,High Risk Precision Score,Low Risk Precision Score,High Risk Recall Score,Low Risk Recall Score,High Risk F1 Score,Low Risk F1 Score
Naive Random Oversampling,0.628533,0.008839,0.996909,0.597701,0.659364,0.017420,0.793741
SMOTE Oversampling,0.641721,0.009267,0.997100,0.620690,0.662753,0.018262,0.796252
Cluster Centroids Undersampling,0.530070,0.005575,0.995670,0.643678,0.416462,0.011054,0.587281
SMOTEENN Hybrid Sampling,0.651763,0.009116,0.997391,0.678161,0.625365,0.017991,0.768734
Balanced Random Forest Ensemble,0.787767,0.035847,0.998139,0.666667,0.908868,0.068035,0.951414


### 2.2.5 Feature Importance

In [29]:
# List the features sorted in descending order by feature importance
features_rank = sorted(zip(myBRFC.feature_importances_, X.columns), reverse=True)
for feature in features_rank:
    print(f"{feature[1]}: ({feature[0]})")

total_rec_prncp: (0.07376667607601396)
total_rec_int: (0.06390324452717588)
total_pymnt_inv: (0.06073336071656837)
total_pymnt: (0.05811195697921674)
last_pymnt_amnt: (0.04951778391272079)
int_rate: (0.02458051789018817)
out_prncp: (0.020398879691407974)
dti: (0.018625883307901298)
max_bal_bc: (0.018378884967316288)
issue_d_Jan-2019: (0.017480030880564042)
installment: (0.01698703048268034)
annual_inc: (0.01661321625916749)
bc_util: (0.01623198470200134)
mths_since_recent_inq: (0.015821923028819504)
total_bc_limit: (0.015385686591272854)
out_prncp_inv: (0.015337070167127763)
avg_cur_bal: (0.01506649550412446)
issue_d_Mar-2019: (0.014926455495755147)
revol_bal: (0.014924489307566175)
mo_sin_old_rev_tl_op: (0.014583210727445232)
total_bal_ex_mort: (0.01428231366275729)
tot_cur_bal: (0.014238473969302518)
mo_sin_old_il_acct: (0.014067369111282934)
total_bal_il: (0.01351439379612201)
bc_open_to_buy: (0.013343855077273033)
il_util: (0.013138998568932102)
tot_hi_cred_lim: (0.0131060052487096

## 2.3 Easy Ensemble AdaBoost Classifier (`EasyEnsembleClassifier`)

### 2.3.1 Train model & predict test target values

In [31]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

# Instantiate the model
myEEC = EasyEnsembleClassifier(random_state=1)

# Train the model using the training data
myEEC.fit(X_train, y_train)

EasyEnsembleClassifier(random_state=1)

In [32]:
# Generate predicted target values
y_pred = myEEC.predict(X_test)

### 2.3.2 Balanced Accuracy Score

In [33]:
# Calculated the balanced accuracy score
modelBAScore = balanced_accuracy_score(y_test, y_pred)
modelBAScore

0.9197970678173006

### 2.3.3 Confusion Matrix

In [34]:
# Display the confusion matrix
myCM = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
myCM_df = pd.DataFrame(
    myCM, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
myCM_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,78,9
Actual low_risk,975,16143


### 2.3.4 Classification Report

In [35]:
# Print the imbalanced classification report
myReport = classification_report_imbalanced(y_test, y_pred, output_dict=True)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.90      0.94      0.14      0.92      0.84        87
   low_risk       1.00      0.94      0.90      0.97      0.92      0.85     17118

avg / total       0.99      0.94      0.90      0.97      0.92      0.85     17205



In [36]:
summary_df.loc['Easy Ensemble AdaBoost'] = [modelBAScore,
                                               myReport['high_risk']['pre'],
                                               myReport['low_risk']['pre'],
                                               myReport['high_risk']['rec'],
                                               myReport['low_risk']['rec'],
                                               myReport['high_risk']['f1'],
                                               myReport['low_risk']['f1']]
summary_df

,Balanced Accuracy Score,High Risk Precision Score,Low Risk Precision Score,High Risk Recall Score,Low Risk Recall Score,High Risk F1 Score,Low Risk F1 Score
Naive Random Oversampling,0.628533,0.008839,0.996909,0.597701,0.659364,0.017420,0.793741
SMOTE Oversampling,0.641721,0.009267,0.997100,0.620690,0.662753,0.018262,0.796252
Cluster Centroids Undersampling,0.530070,0.005575,0.995670,0.643678,0.416462,0.011054,0.587281
SMOTEENN Hybrid Sampling,0.651763,0.009116,0.997391,0.678161,0.625365,0.017991,0.768734
Balanced Random Forest Ensemble,0.787767,0.035847,0.998139,0.666667,0.908868,0.068035,0.951414
Easy Ensemble AdaBoost,0.919797,0.074074,0.999443,0.896552,0.943042,0.136842,0.970424


In [37]:
filepath = Path('Data/Algorithm_Summary.csv')
summary_df.to_csv(filepath)